## Text Classification

In [1]:
from datasets import load_dataset

# Load our data
data = load_dataset("rotten_tomatoes")
data

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [3]:
data['train'][0,-1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

- These short reviews are either labeled as positive (1) or negative (0). This means that we will focus on binary sentiment classification
- BERT, a well-known encoder-only architecture, is a popular choice for creating task-specific
and embedding models.
- In this task we will use RoBERTa model fine-tuned on tweets for sentiment analysis for movie review.

In [5]:
from transformers import pipeline
# Path to our HF model
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
# Load model into pipeline
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda"
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda
/home/nancy/kws_env/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of 

- The model load the tokenizer, which is responsible for converting input text into individual tokens.


![Alt text for the image](images/bert.png)

In [6]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred=[]
for output in tqdm(pipe(KeyDataset(data['test'],'text')),
                  total=len(data['test'])):
    ## output is typically a list of dictionaries, where each dictionary represents a possible label
    ## and its score. For example, [{'label': 'NEGATIVE', 'score': 0.95}, {'label': 'NEUTRAL', 
    ## 'score': 0.03}, {'label': 'POSITIVE', 'score': 0.02}].
    print(output)
    negative_score=output[0]['score']
    positive_score=output[2]['score']
    assignment=np.argmax([negative_score,positive_score])
    y_pred.append(assignment)

  0%|▎                                                                                 | 4/1066 [00:00<02:11,  8.11it/s]

[{'label': 'negative', 'score': 0.005161251872777939}, {'label': 'neutral', 'score': 0.04023365676403046}, {'label': 'positive', 'score': 0.9546050429344177}]
[{'label': 'negative', 'score': 0.007706430274993181}, {'label': 'neutral', 'score': 0.10390972346067429}, {'label': 'positive', 'score': 0.8883838653564453}]
[{'label': 'negative', 'score': 0.7359185218811035}, {'label': 'neutral', 'score': 0.24242739379405975}, {'label': 'positive', 'score': 0.021654037758708}]
[{'label': 'negative', 'score': 0.0030380780808627605}, {'label': 'neutral', 'score': 0.06267133355140686}, {'label': 'positive', 'score': 0.9342905282974243}]
[{'label': 'negative', 'score': 0.11185602098703384}, {'label': 'neutral', 'score': 0.7006853818893433}, {'label': 'positive', 'score': 0.18745863437652588}]


  2%|█▊                                                                               | 24/1066 [00:00<00:20, 50.99it/s]

[{'label': 'negative', 'score': 0.6089192032814026}, {'label': 'neutral', 'score': 0.3732690215110779}, {'label': 'positive', 'score': 0.01781178079545498}]
[{'label': 'negative', 'score': 0.020790275186300278}, {'label': 'neutral', 'score': 0.45805037021636963}, {'label': 'positive', 'score': 0.5211593508720398}]
[{'label': 'negative', 'score': 0.009388203732669353}, {'label': 'neutral', 'score': 0.15621817111968994}, {'label': 'positive', 'score': 0.8343936204910278}]
[{'label': 'negative', 'score': 0.009773931466042995}, {'label': 'neutral', 'score': 0.18585321307182312}, {'label': 'positive', 'score': 0.8043728470802307}]
[{'label': 'negative', 'score': 0.02712375484406948}, {'label': 'neutral', 'score': 0.3788433074951172}, {'label': 'positive', 'score': 0.5940329432487488}]
[{'label': 'negative', 'score': 0.27410057187080383}, {'label': 'neutral', 'score': 0.5992846488952637}, {'label': 'positive', 'score': 0.1266147643327713}]
[{'label': 'negative', 'score': 0.8024442791938782},

  5%|████                                                                             | 53/1066 [00:01<00:10, 93.72it/s]

...

[{'label': 'negative', 'score': 0.9509932994842529}, {'label': 'neutral', 'score': 0.044022247195243835}, {'label': 'positive', 'score': 0.004984575789421797}]
[{'label': 'negative', 'score': 0.21134527027606964}, {'label': 'neutral', 'score': 0.6590261459350586}, {'label': 'positive', 'score': 0.12962861359119415}]
[{'label': 'negative', 'score': 0.8323310017585754}, {'label': 'neutral', 'score': 0.15459269285202026}, {'label': 'positive', 'score': 0.013076309114694595}]
[{'label': 'negative', 'score': 0.767214834690094}, {'label': 'neutral', 'score': 0.22272558510303497}, {'label': 'positive', 'score': 0.010059584863483906}]
[{'label': 'negative', 'score': 0.9139156937599182}, {'label': 'neutral', 'score': 0.07926183938980103}, {'label': 'positive', 'score': 0.006822498980909586}]
[{'label': 'negative', 'score': 0.4860875904560089}, {'label': 'neutral', 'score': 0.48318973183631897}, {'label': 'positive', 'score': 0.030722694471478462}]
[{'label': 'negative', 'score': 0.2312509864568

 98%|████████████████████████████████████████████████████████████████████████████▍ | 1045/1066 [00:08<00:00, 131.06it/s]

[{'label': 'negative', 'score': 0.06191986799240112}, {'label': 'neutral', 'score': 0.8112593293190002}, {'label': 'positive', 'score': 0.12682075798511505}]
[{'label': 'negative', 'score': 0.8747042417526245}, {'label': 'neutral', 'score': 0.11452040821313858}, {'label': 'positive', 'score': 0.010775321163237095}]
[{'label': 'negative', 'score': 0.18314765393733978}, {'label': 'neutral', 'score': 0.7090993523597717}, {'label': 'positive', 'score': 0.10775300115346909}]
[{'label': 'negative', 'score': 0.7373170256614685}, {'label': 'neutral', 'score': 0.24702365696430206}, {'label': 'positive', 'score': 0.015659259632229805}]
[{'label': 'negative', 'score': 0.8234097361564636}, {'label': 'neutral', 'score': 0.16186194121837616}, {'label': 'positive', 'score': 0.01472831517457962}]
[{'label': 'negative', 'score': 0.7951980829238892}, {'label': 'neutral', 'score': 0.16795265674591064}, {'label': 'positive', 'score': 0.03684930503368378}]
[{'label': 'negative', 'score': 0.6539247632026672

100%|██████████████████████████████████████████████████████████████████████████████| 1066/1066 [00:08<00:00, 128.01it/s]

[{'label': 'negative', 'score': 0.13469651341438293}, {'label': 'neutral', 'score': 0.4916401505470276}, {'label': 'positive', 'score': 0.37366339564323425}]
[{'label': 'negative', 'score': 0.9165812134742737}, {'label': 'neutral', 'score': 0.07223008573055267}, {'label': 'positive', 'score': 0.011188692413270473}]
[{'label': 'negative', 'score': 0.6963598132133484}, {'label': 'neutral', 'score': 0.28407543897628784}, {'label': 'positive', 'score': 0.019564734771847725}]
[{'label': 'negative', 'score': 0.9171780943870544}, {'label': 'neutral', 'score': 0.07556598633527756}, {'label': 'positive', 'score': 0.007256021723151207}]
[{'label': 'negative', 'score': 0.4067016541957855}, {'label': 'neutral', 'score': 0.473389208316803}, {'label': 'positive', 'score': 0.11990917474031448}]
[{'label': 'negative', 'score': 0.783981442451477}, {'label': 'neutral', 'score': 0.20310033857822418}, {'label': 'positive', 'score': 0.012918251566588879}]
[{'label': 'negative', 'score': 0.42400074005126953

In [8]:
##evaluation
from sklearn.metrics import classification_report
def evaluate_performance(y_true,y_pred):
    performance=classification_report(
        y_true,y_pred,target_names=['Negative Review','Positive Review']
    )
    print(performance)
evaluate_performance(data['test']['label'],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



![Alt text for the image](images/confusion_matrix.png)

- **Precison**
- Precision: Of all the emails the model predicted as spam, how many were actually spam?
- Precision=TP/(TP+FP)
- A measure of exactness or quality of your positive predictions. High precision means when your model says something is positive, it's very likely correct.
- If your spam filter flagged 10 emails as spam, and 8 of them were actually spam (2 were legitimate emails wrongly flagged), then your precision would be 8/(8+2)=0.8 or 80%.
- When is it crucial? When the cost of a **False Positive** is high.
- Spam detection: You want high precision because you don't want important emails (legitimate) to be marked as spam (false positive).
- Medical diagnosis (for a serious, rare disease): If a positive prediction means invasive and potentially harmful treatment, you want high precision to minimize false alarms.

- **Recall**:
- Of all the emails that were actually spam, how many did the model correctly identify as spam?
- Recall = TP/(TP+FN)
- A measure of completeness or coverage of your positive predictions. High recall means your model caught most of the actual positive cases.
- When the cost of a **False Negative** is high.
- You want high recall because you want to catch as many fraudulent transactions as possible, even if it means flagging some legitimate ones for review (false positives). Missing actual fraud (false negative) is very costly.

- **Accuracy**:
- Number of correctly positive predicted out of all predictions, which indicates the overall correctness of the model.

- **F1 score**
- balances both precision and recall to create a model's overall performance.



## Classification Tasks that Leverage Embeddings

- **Supervised classification**
- Instead of directly using the representation model for classification, we will use an embedding model for generating features. Those features can then be fed into a classifier, thereby creating a two-step approach.
-  A major benefit of this separation is that we do not need to fine-tune our embedding model, which can be costly. In contrast, we can train a classifier, like a logistic regression, on the CPU instead.
-  The model is kept frozen when the embeddings are generated.

In [9]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to embeddings
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [10]:
train_embeddings.shape

(8530, 768)

- This shows that each of our 8,530 input documents has an embedding dimension of 768 and therefore each embedding contains 768 numerical values.
- Then these embedding are fed into the classifier (use logistic regression) 

In [11]:
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression on our train embeddings
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

LogisticRegression(random_state=42)

In [12]:
y_pred=clf.predict(test_embeddings)

In [13]:
evaluate_performance(data['test']['label'],y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



By training a classifier on top of our embeddings, we managed to get an F1 score of 0.85! This demonstrates the possibilities of training a lightweight classifier while keeping the underlying embedding model frozen.

## What If We Do Not Have Labeled Data?